# < Training >

## 목차
## 1. Data Preparation
## 2. Set Parameter for training
## 3. Training

### 1번 부분은 KETI에서 해당 코드가 잘 돌아가는지 확인하려고 만든 Data 가져오기 및 전처리 부분
### 이미 데이터를 준비했다면 2번으로 넘겨도 무방함

In [14]:
# setting
import sys, os
import pandas as pd
import torch

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.clust.ML.tool import meta as ml_meta
from Clust.clust.ML.tool import model as ml_model
from Clust.clust.ML.common import ML_api
from Clust.clust.ML.common import ML_pipeline, tool

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


## Data Preparation (optional) 
### - Data가 준비되었다면 해당 부분을 스킵해도 무방함

### 1-1. Data Parameter Setting

In [15]:
model_purpose = 'regression'
data_name_X = 'regression_energy_cleanLevel4_trainX'
feature_X_list = ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
   'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
   'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
data_name_y = 'regression_energy_cleanLevel4_trainy'
feature_y_list = ['value']
split_mode ="window_split"
data_y_flag = True # 이미 만들어진 Y 데이터를 활용함
scaler_param={
    'scaler_flag':'scale', #scale_param,
    'scale_method' :'minmax',
    'scaler_path' :'./scaler/'
}
transform_param={
    'split_mode' : 'window_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
    #step_split일 경우만 past_step과 future_step이 존재
    'data_clean_option' : False,
    'nan_process_info' :'',
    'max_nan_limit_ratio': ''
}
    
bucket_name = 'integration' 
data_clean_level = 4

### 1-2. Data Ingestion
#### 1-2-1. Select data name 

In [16]:
# Data Ingestion
ingestion_method = 'ms_all'
ingestion_param_X = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_X,
    'feature_list' : feature_X_list                              
}
ingestion_param_y = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_y,
    'feature_list' : feature_y_list                              
}

In [17]:
data_X, data_y = ML_pipeline.Xy_data_preparation(ingestion_param_X, data_y_flag, ingestion_param_y, ingestion_method, db_client)

{'bucket_name': 'integration', 'ms_name': 'regression_energy_cleanLevel4_trainX', 'feature_list': ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']}


#### 1-2-2. Data scaling

In [18]:
dataX_scaled, datay_scaled = ML_pipeline.Xy_data_scaling_train(data_name_X, data_X, data_name_y, data_y, scaler_param)

['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
Make New scaler File
./scaler/regression_energy_cleanLevel4_trainX/minmax/900878d7bba6f2a4017d1c3399909ea7/scaler.pkl
['value']
Make New scaler File


## 1-3. Cleaning and split
### 1-3-1. pipeline - clean low quality column

In [19]:
model_data_clean = False # Front End Parameter 
if model_data_clean:
    nan_process_info = {'type':'num', 'ConsecutiveNanLimit':10, 'totalNaNLimit':100}
    max_nan_limit_ratio = 0.9
    
else:
    nan_process_info = {'type':'num', 'ConsecutiveNanLimit':10000, 'totalNaNLimit':100000}
    max_nan_limit_ratio = 0.5

nan_process_info = {'type':'num', 'ConsecutiveNanLimit':10000, 'totalNaNLimit':100000}
transform_param['nan_process_info'] = nan_process_info
transform_param['max_nan_limit_ratio'] = max_nan_limit_ratio

In [20]:
transform_param

{'split_mode': 'window_split',
 'data_clean_option': False,
 'nan_process_info': {'type': 'num',
  'ConsecutiveNanLimit': 10000,
  'totalNaNLimit': 100000},
 'max_nan_limit_ratio': 0.5}

In [21]:
dataX_scaled = ML_pipeline.clean_low_quality_column(dataX_scaled, transform_param)

### 1-3-2. Train/Val Split pipeline

In [22]:
split_ratio = 0.8

In [23]:
train_X, val_X, train_y, val_y, transform_param = ML_pipeline.split_data_by_mode(dataX_scaled, datay_scaled, split_ratio, transform_param)

{'split_mode': 'window_split', 'data_clean_option': False, 'nan_process_info': {'type': 'num', 'ConsecutiveNanLimit': 10000, 'totalNaNLimit': 100000}, 'max_nan_limit_ratio': 0.5, 'future_step': None, 'past_step': 144}
window_split
None


In [24]:
train_X_array, train_y_array = ML_pipeline.transform_data_by_split_mode(transform_param, train_X, train_y)
val_X_array, val_y_array = ML_pipeline.transform_data_by_split_mode(transform_param, val_X, val_y)

window_size: 144 nan_limit_num: 72
(10944, 24) (72, 144, 24)
(76, 1) (72, 1)
window_size: 144 nan_limit_num: 72
(2736, 24) (19, 144, 24)
(19, 1) (19, 1)


In [25]:
# train, validation의 형태는 np.array

## 2. Set Parameter for training

In [26]:
# 모델 선택
regression_model_list = ['LSTM_rg' , 'GRU_rg',  'CNN_1D_rg', 'LSTM_FCNs_rg',  'FC_rg']
classification_model_list = [ 'LSTM_cf', 'GRU_cf', 'LSTM_cf', 'CNN_1D_cf', 'LSTM_FCNs_cf', 'FC_cf']

model_method = 'LSTM_rg'  # Set model methods i.e., 'LSTM_rg', 'GRU_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg', 'FC_rg' 

### 2-1-1. set model_parameter

In [27]:
# 선택한 모델의 하이퍼 파라미터 정의
setting_model_parameter ={"LSTM_rg":{"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":True},
                     "GRU_rg":{"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":True},
                     "CNN_1D_rg":{"output_channels":64,"kernel_size":3,"stride":1,"dropout":0.1,"padding":0},
                     "LSTM_FCNs_rg":{"num_layers":2,"lstm_dropout":0.4,"fc_dropout":0.1},
                     "FC_rg":{"dropout":2,"bias":0.4}}

model_info = setting_model_parameter[model_method]

# model_parameter 만들기
# model method 선택에 따라 model_parameter 자동 생성
from Clust.clust.ML.common import model_parameter_setting
seq_len, input_size = train_X_array.shape[1], train_X_array.shape[2]
model_parameter = model_parameter_setting.get_regression_model_parameter(model_method, model_info, seq_len, input_size)

{'hidden_size': 64, 'num_layers': 2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': True}


### 2-1-1. set train_parameter

In [28]:
train_parameter = {"lr":1e-4,"weight_decay":1e-6,"n_epochs":100,"batch_size":16, "device": device}

### 2.2 Set Model name and path pipeline

In [29]:
# 임의로 정한 name 및 path --> 사용자 마음대로 수정 가능
model_name = 'regression_model_test'
model_path_list = ['model',model_name]
model_file_path = ml_model.get_model_file_path(model_path_list, model_method)

./Models/LSTM_rg/model/regression_model_test/model.pkl


## 3.Training

In [30]:
from Clust.clust.ML.regression.train import RegressionTrain as RML

rml = RML()
rml.set_param(train_parameter)
rml.set_model(model_method, model_parameter)
rml.set_data(train_X_array, train_y_array, val_X_array, val_y_array)
rml.train()
rml.save_best_model(model_file_path)

Start training model
[1/100] Training loss: 0.2152	 Validation loss: 0.1540
[2/100] Training loss: 0.1981	 Validation loss: 0.1418
[3/100] Training loss: 0.1745	 Validation loss: 0.1300
[4/100] Training loss: 0.1777	 Validation loss: 0.1185
[5/100] Training loss: 0.1632	 Validation loss: 0.1072
[6/100] Training loss: 0.1450	 Validation loss: 0.0962
[7/100] Training loss: 0.1329	 Validation loss: 0.0855
[8/100] Training loss: 0.1143	 Validation loss: 0.0751
[9/100] Training loss: 0.0915	 Validation loss: 0.0653
[10/100] Training loss: 0.0948	 Validation loss: 0.0561
[50/100] Training loss: 0.0379	 Validation loss: 0.0331
[100/100] Training loss: 0.0346	 Validation loss: 0.0332

Training complete in 0m 10s


In [31]:
train_parameter

{'lr': 0.0001,
 'weight_decay': 1e-06,
 'n_epochs': 100,
 'batch_size': 16,
 'device': 'cpu'}

In [32]:
model_parameter

{'rnn_type': 'lstm',
 'input_size': 24,
 'hidden_size': 64,
 'num_layers': 2,
 'output_dim': 1,
 'dropout': 0.1,
 'bidirectional': True}

In [33]:
transform_param

{'split_mode': 'window_split',
 'data_clean_option': False,
 'nan_process_info': {'type': 'num',
  'ConsecutiveNanLimit': 10000,
  'totalNaNLimit': 100000},
 'max_nan_limit_ratio': 0.5,
 'future_step': None,
 'past_step': 144}